In [5]:
import requests
import re

data = {
    "temperature": 0.0,
    "max_tokens": 4096,
    "messages": [{
        "role": "user",
        "content": [{
            "type": "text",
            "text": '''You are GPT-4o. Your task is to evaluate a Query Assistant Response—generated by a Qwen-2.5-7B base model—against the Task Generation Success Criteria defined in the Query Assistant Prompt.
            
Please evaluate whether the Query Assistant Response fulfills ALL of these criteria. Show your step-by-step reasoning inside <think> and </think> tags, then provide your final decision inside <answer> and </answer> tags as a single word: "yes" if it meets the criteria or "no" if it does not.

<think>
[Provide your detailed evaluation and reasoning here]
</think>
<answer>
[yes/no]
</answer>'''
        }, {
            "type": "text",
            "text": '''QUERY ASSISTANT PROMPT: A Policy Assistant Goal set by a User, and a game between a Query Assistant and a Policy Assistant. 
                    
The User gives the Policy Assistant Goal and tasks the Query Assistant to generate a query to challenge and help the Policy Assistant better reach the Policy Assistant Goal, then the Policy Assistant tries to solve the Query Assistant's generated query, then the Query Assistant generates another query, then the Policy Assistant tries to solve that query, etc. After each Query Assistant response, there is a Query Average Success score that signals the query's difficulty to the Policy Assistant, where 0.0 means the query is currently too difficult or impossible for the Policy Assistant and 1.0 means the query is currently too easy for the Policy Assistant. After each Policy Assistant response, there is Policy Reward where 0.0 means that the Policy Assistant was not able to reach the same target answer proposed by the Query Assistant and 1.0 means that the Policy Assistant was able to reach the same target answer. The Query and Policy Assistant both first think about the reasoning process in the mind and then provides the Policy or Query Assistant with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e. your response format should be: <think> reasoning process here </think>

<answer> answer here </answer>. User: You must put your answer inside <answer> </answer> tags, i.e., <answer> answer here </answer>.

This is the problem:
Given the Policy Assistant Goal, Game History of up to the last 3 Query Assistant Answer and Average Success and Policy Assistant Answer and Reward turns, Task Generation Success Criteria, and Example Final Answer Format, generate a task that fulfills the task generation success criteria. 


# Policy Assistant Goal: 
Get really good at math.


# Game History:

**Turn 1**
Query Assistant Answer: {
    "prompt": "If the radius of a circle is 30, what is its area?",
    "target": 2827.43,
    "solution": "The area of a circle is given by the formula A = πr^2. Given a radius of 30, the area is calculated by squaring the radius, 30^2 = 900, and then multiplying by π, approximately 3.14, therefore A = 2827.43 square units."
}

Query Average Success: 0.75

Policy Assistant Answer: 2827.43

Policy Reward: 1.0

**Turn 2**
Query Assistant Answer: {
    "prompt": "What is the circumference of a circle with a diameter of 14?",
    "target": 43.98,
    "solution": "The circumference of a circle can be calculated using the formula C = πd, where d is the diameter. Given a diameter of 14, we multiply by π (approximately 3.14), yielding a circumference of 43.98 units."
}

Query Average Success: 0.38

Policy Assistant Answer: 43.98

Policy Reward: 1.0

**Turn 3**
Query Assistant Answer: {
    "prompt": "If the radius of a circle is 7, what is its circumference?",
    "target": 43.98,
    "solution": "The circumference of a circle can be calculated using the formula C = 2πr, where r is the radius. Given a radius of 7, we multiply by 2 times π (approximately 3.14), resulting in a circumference of 43.98 units."
}

Query Average Success: 0.75

Policy Assistant Answer: 43.98

Policy Reward: 1.0


# Task Generation Success Criteria: The generated task should be:

1) maximizing user and Policy Assistant benefit and helpfulness and minimizing user and Policy Assistant risk, harm, and unwanted outcomes

2) novel - avoid designing a task that is drawn from the same linguistic templates or differ only in superficial details

3) difficult yet tractable - the task should be challenging for the Policy Assistant to complete, without being completely intractable. A Query Average Success score of 0.5 is ideal.

4) clear - the task should be specific enough to not require further user input to understand what is being asked of the Policy Assistant and should provide all the context and detail necessary. The task and final answer must be a dictionary with 'prompt', 'target', and 'solution' keys. The 'prompt' should be a math problem, the 'target' should be the expected answer to the math problem as an int or float, and the 'solution' must be a solution to the math problem that clearly and unambiguously leads to the expected answer 'target'.

                    
# Example Query Assistant Reasoning and Final Answer Response Format: <think>
reasoning process here
</think>
<answer>
{
    'prompt': 'What is 1 + 1?',
    'target': 2,
    'solution': 'To solve 1 + 1, we add the two numbers together. Starting with the number 1, adding another 1 gives us 2. Therefore, the answer is 2.'
}
</answer>


Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> answer here </answer>. Think step by step inside <think> tags.
Query Assistant: <think>''',
        }, {
            "type": "text",
            "text": '''QUERY ASSISTANT RESPONSE: When creating a task that maximizes user and Policy Assistant benefit and helpfulness and minimizes user and Policy Assistant risk, harm, and unwanted outcomes, I must ensure that the task is clear, novel, difficult yet tractable, and aligned with the Policy Assistant's goal of getting really good at math. To follow the success criteria, the final answer must be a dictionary with 'prompt', 'target', and 'solution' keys. The 'prompt' should be a math problem, the 'target' should be the expected answer to the math problem, and the 'solution' must be a clear and unambiguous solution to the math problem.</think>\n<answer>\n{\n    "prompt": "What is the value of $9901\\cdot101-99\\cdot10101?$",\n    "target": 2,\n    "solution": "The likely fastest method will be direct computation. $9901\\cdot101$ evaluates to $1000001$ and $99\\cdot10101$ evaluates to $999999$. The difference is \\boxed{\\textbf{(A) }2}."\n}\n</answer>'''
        }]
    }],
    "stop": ["<|im_end|>"],
    "n": 1,
    "model": "dev-gpt-4o-vision-2024-05-13"
}

url = "https://skilled-chigger-mentally.ngrok-free.app/api/send_request"
response = requests.post(url, json=data)
print(response)

# Print the full response content
response_content = response.json()['choices'][0]['message']['content']
print("Full Response Content:")
print(response_content)

# Extract the text between the <answer> tags using a regular expression
match = re.search(r'<answer>\s*(.*?)\s*</answer>', response_content, re.DOTALL)
if match:
    extracted_answer = match.group(1).strip()
    # Optionally, if you expect only 'yes' or 'no' you can enforce that:
    decision = extracted_answer.lower()
    if decision in ['yes', 'no']:
        print("Extracted decision:", decision)
    else:
        print("Extracted answer does not contain a simple 'yes' or 'no':")
        print(extracted_answer)
else:
    print("No <answer> tags found in the response.")


<Response [200]>
Full Response Content:
<think>
To evaluate the Query Assistant Response against the Task Generation Success Criteria, I will analyze each criterion step-by-step:

1. **Maximizing user and Policy Assistant benefit and helpfulness and minimizing user and Policy Assistant risk, harm, and unwanted outcomes**:
   - The task is a math problem that aligns with the Policy Assistant's goal of getting really good at math. It does not pose any risk or harm and is purely educational.

2. **Novel - avoid designing a task that is drawn from the same linguistic templates or differ only in superficial details**:
   - The task is novel compared to the previous tasks in the game history. It involves a different type of math problem (arithmetic with large numbers) rather than geometry (circles).

3. **Difficult yet tractable - the task should be challenging for the Policy Assistant to complete, without being completely intractable. A Query Average Success score of 0.5 is ideal**:
   - Th